In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3
import os
import numpy as np
from requests_html import HTMLSession
from requests_file import FileAdapter

session = HTMLSession()

session.mount('file://',FileAdapter())
pwd = os.getcwd().replace("\\","/")
html_obj = session.get(f'file:///{pwd}/台股公告 _ Anue鉅亨.html')

soup = BeautifulSoup(html_obj.text,"html.parser")

In [2]:
titletime = []
id = []
idBefore = []
content = []
  

# # 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# # 內容============================
# titles = soup.select('._1xc2')
# for title in titles:
#   head,mid,tail = title.find('h3').text.partition(':')
#   id.append(head)
#   content.append(title.text)
# # print(len(id))
# # print(len(content))

# 內容===============================================================================================================
titles = soup.select('._1xc2')
# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
  content.append(title.text)
# 2.去除【月營收】
for x in idBefore:
  x2 = x.find('月營收')
  if x2 < 0: # 沒有【月營收】
    id.append(x)
  else: # 有【月營收】
    x3 = x2 - 1
    x4 = x[:x3]
    id.append(x4)
# =================================================================================================================


data = {
  'date': titletime,
  'code': id,
  'content': content
}


df = pd.DataFrame(data)
# print(df)

# # 今日新闻
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']=='2022-08-31'            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]
# print(filtered_df)


# connecting to sqlite
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()
uniqueSql = '''select * from news t'''


for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
  except:
    pass

# print(filtered_df)

# 確認當日新聞 ========================================================================
dfToday = pd.read_sql(uniqueSql,conn)
# df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
df_maskToday = dfToday['date']=='2022-08-31'           # 日期為今天的存檔
positionsToday = np.flatnonzero(df_maskToday)
filtered_dfToday=dfToday.iloc[positionsToday]
# ======================================================================================

filtered_dfToday

,date,code,content
1706,2022-08-31,愛山林,愛山林:代取得人依證交法第43條之1第1項取得本公司股份變動達 1%公告
1707,2022-08-31,樂威科-KY,樂威科-KY:依證交法第43條之1第1項規定，代非公開發行公司辦理公告
1708,2022-08-31,正文,正文:本公司將收回已發行之限制員工權利新股辦理註銷完成公告(補充公告)
1709,2022-08-31,正文,正文:正文科技股份有限公司（公司代號：4906）註銷收回股票事宜。
1710,2022-08-31,正文,正文:正文科技股份有限公司（公司代號：4906）註銷買回股票事宜。
...,...,...,...
1860,2022-08-31,矽格,矽格:對於經濟日報C4報導之說明
1861,2022-08-31,訊達電腦,訊達電腦:公告本公司111年07月份自結合併財務報告之流動比率、速動比率及負債比率
1862,2022-08-31,互億,互億:互億公司股東會決議通過撤銷股票公開發行案(增加其他應敘明事項)
1863,2022-08-31,福邦證,福邦證:本公司110年度盈餘轉增資發行新股公告


In [1]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT UNIQUE, 
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================

In [2]:
from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()

data.airports()

ModuleNotFoundError: No module named 'colab'